In [1]:
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from sgp4.api import Satrec
from sgp4.ext import invjday
from astropy.time import Time
from astropy import units as u
from sgp4.conveniences import jday
from joblib import Parallel, delayed
from astropy.coordinates import ITRS
from datetime import datetime, timedelta
from astropy.coordinates import TEME, CartesianDifferential, CartesianRepresentation

In [3]:
username = 'asas4539@hanyang.ac.kr'
password = 'onsaemiro1729!!'

In [18]:
def get_tle(sat_num):
    # Space-Track API의 URL
    url = "https://www.space-track.org/ajaxauth/login"
    payload = {"identity": username, "password": password}

    session = requests.Session()
    response = session.post(url, data=payload)
    if response.status_code != 200:
        print("로그인에 실패했습니다.")
        return None
    response = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/tle/NORAD_CAT_ID/{sat_num}/EPOCH/2022-12-31--2024-08-01/orderby/EPOCH%20asc/format/3le/emptyresult/show"
    )

    return response

In [19]:
def get_kvn(sat_num):
    # Space-Track API의 URL
    url = "https://www.space-track.org/ajaxauth/login"
    payload = {"identity": username, "password": password}

    session = requests.Session()
    response = session.post(url, data=payload)
    if response.status_code != 200:
        print("로그인에 실패했습니다.")
        return None
    response = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{sat_num}/EPOCH/2022-12-31--2024-08-01/orderby/EPOCH%20asc/format/kvn/emptyresult/show"
    )

    return response

In [20]:
response = get_created_date(42984)
tle_datas = response.text.split('\r\n')

In [21]:
import re

creation_dates = re.findall(r"CREATION_DATE\s*=\s*([\d\-T:]+)", response.text)

In [22]:
creation_dates

['2022-12-31T09:06:13',
 '2022-12-31T18:47:40',
 '2023-01-01T17:55:23',
 '2023-01-01T16:36:13',
 '2023-01-02T02:38:49',
 '2023-01-02T17:42:09',
 '2023-01-03T02:40:54',
 '2023-01-03T00:46:15',
 '2023-01-04T03:14:03',
 '2023-01-03T16:36:13',
 '2023-01-04T18:02:30',
 '2023-01-04T16:46:13',
 '2023-01-05T05:38:49',
 '2023-01-05T18:16:45',
 '2023-01-07T03:04:56',
 '2023-01-06T15:46:13',
 '2023-01-07T07:16:16',
 '2023-01-07T17:35:56',
 '2023-01-08T02:49:26',
 '2023-01-08T07:36:13',
 '2023-01-08T18:29:57',
 '2023-01-09T04:15:35',
 '2023-01-09T07:46:14',
 '2023-01-09T19:07:07',
 '2023-01-10T05:52:54',
 '2023-01-10T10:36:13',
 '2023-01-10T20:00:10',
 '2023-01-11T03:21:39',
 '2023-01-11T17:22:06',
 '2023-01-11T16:06:14',
 '2023-01-12T02:25:51',
 '2023-01-12T23:26:13',
 '2023-01-13T04:28:49',
 '2023-01-13T20:32:32',
 '2023-01-15T01:06:11',
 '2023-01-15T21:11:06',
 '2023-01-16T02:24:46',
 '2023-01-16T07:56:12',
 '2023-01-16T17:40:19',
 '2023-01-17T02:22:43',
 '2023-01-17T18:28:45',
 '2023-01-17T07:

In [23]:
def get_info(change_df):
    all_info = []
    earth_radius = 6378.137
    rad2deg = 180.0 / 3.141592653589793

    for i in range(len(change_df) - 1):
        sat_info = change_df.iloc[i]
        next_sat_info = change_df.iloc[i + 1]

        tle_1, tle_2 = sat_info['first_line'], sat_info['second_line']
        satellite = Satrec.twoline2rv(tle_1, tle_2)
        jdsatepoch = satellite.jdsatepoch
        jdsatepochfrac = satellite.jdsatepochF
        epochdatetime = list(invjday(jdsatepoch + jdsatepochfrac))
        epochdatetime[-1] = int(epochdatetime[-1])

        next_tle_1, next_tle_2 = next_sat_info['first_line'], next_sat_info['second_line']
        next_satellite = Satrec.twoline2rv(next_tle_1, next_tle_2)
        next_jdsatepoch = next_satellite.jdsatepoch
        next_jdsatepochfrac = next_satellite.jdsatepochF
        next_epochdatetime = list(invjday(next_jdsatepoch + next_jdsatepochfrac))
        next_epochdatetime[-1] = int(next_epochdatetime[-1])

        start = datetime(*epochdatetime)
        end = datetime(*next_epochdatetime)
        end += timedelta(minutes=1)
        epoch = start
        jd_lst = []
        fr_lst = []
        time_lst = []

        while (epoch < end):
            year = epoch.year
            month = epoch.month
            day = epoch.day
            hour = epoch.hour
            minute = epoch.minute
            second = epoch.second
            jd, fr = jday(year, month, day, hour, minute, second)

            time_lst.append(pd.to_datetime(epoch))
            jd_lst.append(jd)
            fr_lst.append(fr)
            epoch += timedelta(minutes=1)

        e, r, v = satellite.sgp4_array(np.array(jd_lst), np.array(fr_lst))
        t_lst = Time(list(np.array(jd_lst) + np.array(fr_lst)), format='jd')
        teme_p = CartesianRepresentation(r[:, 0] * u.km, r[:, 1] * u.km, r[:, 2] * u.km)
        teme_v = CartesianDifferential(v[:, 0] * u.km / u.s, v[:, 1] * u.km / u.s, v[:, 2] * u.km / u.s)
        teme = TEME(teme_p.with_differentials(teme_v), obstime=t_lst)
        itrs_geo = teme.transform_to(ITRS(obstime=t_lst))
        locations = itrs_geo.earth_location
        geodetic_coords = locations.geodetic

        segment_df = pd.DataFrame({
            'time': time_lst,
            'x': r[:, 0], 'y': r[:, 1], 'z': r[:, 2],
            'vx': v[:, 0], 'vy': v[:, 1], 'vz': v[:, 2],
            'x_earth': teme_p.x.value, 'y_earth': teme_p.y.value, 'z_earth': teme_p.z.value,
            'vx_earth': teme_v.d_x.value, 'vy_earth': teme_v.d_y.value, 'vz_earth': teme_v.d_z.value,
            'altitude': (r[:, 0] ** 2 + r[:, 1] ** 2 + r[:, 2] ** 2) ** 0.5,
            'velocity': (v[:, 0] ** 2 + v[:, 1] ** 2 + v[:, 2] ** 2) ** 0.5,
            'apogee': [satellite.alta * earth_radius] * len(time_lst),
            'perigee': [satellite.altp * earth_radius] * len(time_lst),
            'inclination': [satellite.inclo * rad2deg] * len(time_lst),
            'eccentricity': [satellite.ecco] * len(time_lst),
            'raan': [np.deg2rad(satellite.nodeo)] * len(time_lst),
            'longitude': geodetic_coords.lon.value,
            'latitude': geodetic_coords.lat.value,
            'height': geodetic_coords.height.value,
        })
        all_info.append(segment_df)

    result = pd.concat(all_info).reset_index(drop=True)
    result['time'] = pd.to_datetime(result['time'])
    result = result.set_index('time')
    result.sort_index(inplace=True)

    return result

In [24]:
number_lst = [39227, 44343, 44349, 44350, 44351, 44353, 44358, 58464, 46267, 53611, 53019, 51961, 47775, 51969, 58722, 56361,
              56289, 56783, 48018, 43823, 45246, 29349, 37265, 42691, 42984, 55841]
number_lst = [58400]
to_do_lst = []
number_lst = [42984]

In [25]:
chande_dict={}

In [26]:
for number in number_lst:
    print(number)
    # TLE_df 생성
    response = get_tle(number)
    tle_datas = response.text.split('\r\n')
    tle_first_lst = []
    tle_second_lst = []
    for i in range(0, len(tle_datas) - 2, 3):
        tle_first_lst.append(tle_datas[i + 1])
        tle_second_lst.append(tle_datas[i + 2])
    tle_df = pd.DataFrame({'first_line': tle_first_lst, 'second_line': tle_second_lst})
    tle_df.drop_duplicates(keep='first', inplace=True)
    tle_df.reset_index(inplace=True, drop=True)

    # change_df 생성
    change_times = []
    for i in range(len(tle_df)):
        tle_1, tle_2 = tle_df['first_line'].iloc[i], tle_df['second_line'].iloc[i]
        satellite = Satrec.twoline2rv(tle_1, tle_2)
        jdsatepoch = satellite.jdsatepoch
        jdsatepochfrac = satellite.jdsatepochF
        epochdatetime = invjday(jdsatepoch + jdsatepochfrac)
        change_times.append(epochdatetime)
    change_df = pd.DataFrame(change_times)
    change_df.columns = ['year', 'month', 'day', 'hour', 'minute', 'second']
    change_df['first_line'] = tle_df['first_line']
    change_df['second_line'] = tle_df['second_line']
    change_df['time'] = pd.to_datetime(change_df[['year', 'month', 'day', 'hour', 'minute', 'second']])
    change_df['time'].drop_duplicates(keep='first', inplace=True)
    
    chande_dict[number]=change_df['time']

    # end point 삽입 및 day_change_idx 추출
    end_point = change_df.iloc[-1:].copy()
    end_point['month'] = 8
    end_point['day'] = 1
    end_point['hour'] = 0
    end_point['minute'] = 0
    end_point['second'] = 0
    end_point['time'] = pd.to_datetime(end_point[['year', 'month', 'day', 'hour', 'minute', 'second']])
    change_df = pd.concat([change_df, end_point], axis=0).copy()
    change_df.reset_index(inplace=True, drop=True)
    day_change_idx = list(change_df[['year', 'month', 'day']].drop_duplicates().index)

    # 병렬처리
    seg_lst = []
    for i in range(len(day_change_idx) - 1):
        hour_idx = change_df.iloc[day_change_idx[i]:day_change_idx[i + 1] + 1, :4].drop_duplicates().index
        seg_lst.append(change_df.iloc[hour_idx])
    result_lst = Parallel(n_jobs=-1)(delayed(get_info)(seg) for seg in tqdm(seg_lst))
    result_df = pd.concat(result_lst).sort_values(by=['time'])

    print(f'Saving {number}...')
    result_df.to_csv(f'Database/{number}_new.csv')
    print(f'Saved {number}!\n')

42984


100%|██████████| 549/549 [00:24<00:00, 22.55it/s]


Saving 42984...
Saved 42984!


In [27]:
# for i in number_lst:
#     chande_dict[i].to_csv(f'Database/{i}_tle_change_info.csv')

In [28]:
change_df

,year,month,day,hour,minute,second,first_line,second_line,time
0,2022,12,31,1,27,42.796801,1 42984U 17067A 22365.06091200 -.00000352 0...,2 42984 0.0334 82.6392 0001326 181.7941 330...,2022-12-31 01:27:42.796801329
1,2022,12,31,10,31,42.478181,1 42984U 17067A 22365.43868609 -.00000351 0...,2 42984 0.0344 83.1290 0001351 181.2411 106...,2022-12-31 10:31:42.478180826
2,2023,1,1,6,6,42.746679,1 42984U 17067A 23001.25466142 -.00000349 0...,2 42984 0.0366 84.4783 0001385 182.2738 38...,2023-01-01 06:06:42.746678889
3,2023,1,1,6,6,42.746679,1 42984U 17067A 23001.25466142 -.00000349 0...,2 42984 0.0366 84.4783 0001385 182.2738 38...,2023-01-01 06:06:42.746678889
4,2023,1,1,14,34,25.699290,1 42984U 17067A 23001.60724189 -.00000349 0...,2 42984 0.0375 84.7122 0001389 183.5163 164...,2023-01-01 14:34:25.699289739
...,...,...,...,...,...,...,...,...,...
1321,2024,7,30,14,54,21.593668,1 42984U 17067A 24212.62108326 -.00000364 0...,2 42984 0.0272 76.7826 0000780 74.0486 134...,2024-07-30 14:54:21.593668163
1322,2024,7,30,23,22,21.770969,1 42984U 17067A 24212.97386309 -.00000365 0...,2 42984 0.0282 77.1089 0000771 76.1046 259...,2024-07-30 23:22:21.770969332
1323,2024,7,31,6,9,33.116264,1 42984U 17067A 24213.25663329 -.00000366 0...,2 42984 0.0288 77.5259 0000778 76.4671 0...,2024-07-31 06:09:33.116263747
1324,2024,7,31,20,32,42.740725,1 42984U 17067A 24213.85605024 -.00000367 0...,2 42984 0.0169 34.5896 0000561 108.7522 227...,2024-07-31 20:32:42.740725279


In [32]:
change_df=change_df.iloc[:-1]

In [35]:
change_df['created_date']=np.array(creation_dates)

C:\Users\asas4\AppData\Local\Temp\ipykernel_12680\923531360.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  change_df['created_date']=np.array(creation_dates)


In [37]:
change_df

,year,month,day,hour,minute,second,first_line,second_line,time,created_date
0,2022,12,31,1,27,42.796801,1 42984U 17067A 22365.06091200 -.00000352 0...,2 42984 0.0334 82.6392 0001326 181.7941 330...,2022-12-31 01:27:42.796801329,2022-12-31T09:06:13
1,2022,12,31,10,31,42.478181,1 42984U 17067A 22365.43868609 -.00000351 0...,2 42984 0.0344 83.1290 0001351 181.2411 106...,2022-12-31 10:31:42.478180826,2022-12-31T18:47:40
2,2023,1,1,6,6,42.746679,1 42984U 17067A 23001.25466142 -.00000349 0...,2 42984 0.0366 84.4783 0001385 182.2738 38...,2023-01-01 06:06:42.746678889,2023-01-01T17:55:23
3,2023,1,1,6,6,42.746679,1 42984U 17067A 23001.25466142 -.00000349 0...,2 42984 0.0366 84.4783 0001385 182.2738 38...,2023-01-01 06:06:42.746678889,2023-01-01T16:36:13
4,2023,1,1,14,34,25.699290,1 42984U 17067A 23001.60724189 -.00000349 0...,2 42984 0.0375 84.7122 0001389 183.5163 164...,2023-01-01 14:34:25.699289739,2023-01-02T02:38:49
...,...,...,...,...,...,...,...,...,...,...
1320,2024,7,29,21,19,8.167862,1 42984U 17067A 24211.88828898 -.00000363 0...,2 42984 0.0253 75.7241 0000785 71.6777 233...,2024-07-29 21:19:08.167861998,2024-07-30T03:13:00
1321,2024,7,30,14,54,21.593668,1 42984U 17067A 24212.62108326 -.00000364 0...,2 42984 0.0272 76.7826 0000780 74.0486 134...,2024-07-30 14:54:21.593668163,2024-07-30T17:34:11
1322,2024,7,30,23,22,21.770969,1 42984U 17067A 24212.97386309 -.00000365 0...,2 42984 0.0282 77.1089 0000771 76.1046 259...,2024-07-30 23:22:21.770969332,2024-07-31T03:53:23
1323,2024,7,31,6,9,33.116264,1 42984U 17067A 24213.25663329 -.00000366 0...,2 42984 0.0288 77.5259 0000778 76.4671 0...,2024-07-31 06:09:33.116263747,2024-07-31T17:21:52
